In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Missing").getOrCreate()

In [19]:
training_df = spark.read.csv("test1based.csv", header = True, inferSchema = True)
training_df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [20]:
training_df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [21]:
training_df.columns

['Name', 'age', 'Experience', 'Salary']

In [23]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols = ["age", "Experience"], outputCol = "independent Features")

In [24]:
output = featureassembler.transform(training_df)

In [25]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [26]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'independent Features']

In [29]:
final_df = output.select("independent Features", "Salary")

In [68]:
final_df.show()

+--------------------+------+
|independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [69]:
from pyspark.ml.regression import LinearRegression
# Train test split 
train_data, test_data = final_df.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = "independent Features", labelCol = "Salary")
regressor = regressor.fit(train_data)


In [70]:
# Coefficients
regressor.coefficients

DenseVector([-263.7076, 1767.624])

In [71]:
# Intercepts
regressor.intercept

19919.060052212404

In [72]:
# Prediction
pred_results = regressor.evaluate(test_data)

In [73]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [29.0,4.0]| 20000|19342.03655352618|
+--------------------+------+-----------------+



In [74]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(657.9634464738192, 432915.89689570636)